# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
citiesFile = pd.read_csv("../output_data/cities.csv",index_col = 0)
citiesFile


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,port hawkesbury,45.6169,-61.3485,56.59,94,100,6.91,CA,1665015594
1,east london,-33.0153,27.9116,64.67,83,100,7.34,ZA,1665015410
2,hambantota,6.1241,81.1185,78.15,82,100,16.69,LK,1665015594
3,punta arenas,-53.1500,-70.9167,44.71,76,75,13.80,CL,1665015595
4,hilo,19.7297,-155.0900,83.25,79,100,5.99,US,1665015596
...,...,...,...,...,...,...,...,...,...
566,marabba,12.3500,32.1833,75.60,72,91,9.10,SD,1665016626
567,fougamou,-1.2154,10.5838,73.18,98,100,2.37,GA,1665016627
568,nyagan,62.1406,65.3936,41.09,94,96,4.34,RU,1665016329


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Lat and Lng as locations and Humidity as the weight
locations = citiesFile[["Lat", "Long"]]
humidity = citiesFile["Humidity"].astype(float)

In [19]:
# Show Heatmap
fig = gmaps.figure(center=(0,0), zoom_level=2)

# Add heat layer
heatLayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heatLayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# Find cities with humidity less than 30 and cloudiness to 0 and tmeperature between 60 to 80 and

cleanCityDF = citiesFile.loc[(citiesFile["Humidity"] <= 60) & (citiesFile["Cloudiness"] <=10) & \
                                   (citiesFile["Max Temp"] >= 60) & (citiesFile["Max Temp"] <= 85)].dropna()

cleanCityDF

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
8,gra liyia,35.0167,25.6833,70.12,50,1,11.99,GR,1665015598
39,broken hill,-31.9500,141.4333,75.13,30,0,10.36,AU,1665015452
63,hamilton,39.1834,-84.5333,65.25,59,0,3.44,US,1665015661
90,agadez,19.7500,10.2500,83.26,22,4,14.16,NE,1665015765
168,miles city,46.4083,-105.8406,72.57,43,0,8.05,US,1665015870
174,yulara,-25.2406,130.9889,76.95,19,0,3.80,AU,1665015873
178,buraydah,26.3260,43.9750,72.05,17,0,5.48,SA,1665015935
231,laredo,27.5064,-99.5075,84.49,48,0,3.44,US,1665016022
241,port augusta,-32.5000,137.7667,74.39,41,0,9.24,AU,1665016028


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Create a hotel DF
hotelDF = cleanCityDF.loc[:,["City", "Country", "Lat", "Long"]]
hotelDF["Hotel Name"] = ""
hotelDF


,City,Country,Lat,Long,Hotel Name
City_ID,,,,,
8,gra liyia,GR,35.0167,25.6833,
39,broken hill,AU,-31.9500,141.4333,
63,hamilton,US,39.1834,-84.5333,
90,agadez,NE,19.7500,10.2500,
168,miles city,US,46.4083,-105.8406,
174,yulara,AU,-25.2406,130.9889,
178,buraydah,SA,26.3260,43.9750,
231,laredo,US,27.5064,-99.5075,
241,port augusta,AU,-32.5000,137.7667,


In [22]:
# set base URL and parameters
parameters = {
    "type" : "hotel",
    "radius" : 5000,
    "key" : g_key    
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# create loop to search for the hotels
for index, row in hotelDF.iterrows():
    lat = row["Lat"]
    lng = row["Long"]
    cityName = row["City"]
    
    # Change target coordinates each iteration while leaving other params in place
    parameters["location"] = f"{lat}, {lng}"
    
    # Request and jsonify
    print(f"Retrieving Results for Index {index}: {cityName}.")
    response = requests.get(base_url, params=parameters).json()
    
    # get results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Nearest hotel in {cityName} is {results[0]['name']}.")
        hotelDF.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("No Results....skipping......")
        
    print("------------------------------")
        
print("------------------------------------------")

Retrieving Results for Index 8: gra liyia.
No Results....skipping......
------------------------------
Retrieving Results for Index 39: broken hill.
No Results....skipping......
------------------------------
Retrieving Results for Index 63: hamilton.
No Results....skipping......
------------------------------
Retrieving Results for Index 90: agadez.
No Results....skipping......
------------------------------
Retrieving Results for Index 168: miles city.
No Results....skipping......
------------------------------
Retrieving Results for Index 174: yulara.
No Results....skipping......
------------------------------
Retrieving Results for Index 178: buraydah.
No Results....skipping......
------------------------------
Retrieving Results for Index 231: laredo.
No Results....skipping......
------------------------------
Retrieving Results for Index 241: port augusta.
No Results....skipping......
------------------------------
Retrieving Results for Index 271: glendive.
No Results....skippin

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelDF.iterrows()]
locations = hotelDF[["Lat", "Long"]]

In [25]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(marker)
# Display figure
fig



AttributeError: module 'collections' has no attribute 'Iterable'